In [1]:
# import packages and set options
from formatting_functions_open_source import *
from utility_functions import *
import numpy as np
import pandas as pd
import xlsxwriter
import pandas.io.formats.excel

# this allows custom formatting of header
pandas.io.formats.excel.ExcelFormatter.header_style = None


In [2]:
# load in data
twoD_df = pd.read_csv('Data/avg_medical_svc_cost.csv')

threeD_row_df_1 = pd.read_csv('Data/clients_by_gender.csv')

threeD_row_df_2 = pd.read_csv('Data/clients_by_gender_2.csv')

test_dtype_df = pd.read_csv('Data/test_set_datatype.csv')


In [3]:
# set indices
twoD_df.set_index('Department', inplace=True)

threeD_row_df_1.set_index(['Department', 'Gender'], inplace=True)

threeD_row_df_2.set_index(['Department','Gender','Trans'], inplace=True)

test_dtype_df.set_index('Department', inplace=True)

In [4]:
test_dtype_df['date_todt'] = pd.to_datetime(test_dtype_df['date_todt'])
test_dtype_df['date_alt_todt'] = pd.to_datetime(test_dtype_df['date_alt_todt'])
test_dtype_df['datetime_todt'] = pd.to_datetime(test_dtype_df['datetime_todt'])
test_dtype_df['datetime_alt_todt'] = pd.to_datetime(test_dtype_df['datetime_alt_todt'])

In [16]:
len(test_dtype_df.date.name)

4

In [6]:
def set_col_data_type(df, wb, sheet, col_name, data_type, col_width_method=None, column_offset=0):

    # This function will apply the specified formatting to the specified column
    ## Meant only for dataframes that have the same data type for ALL non-index columns, but can have any number of columns and indices
    ### Note: this will set ALL data columns to the same width!

    # ARGUMENTS
    
    ## MANDATORY:
    ### df is your data from your dataframe
    ### wb is your workbook
    ### sheet is your worksheet
    ### col_name is the name of your column
    ### data_type is the type of numeric data:
    #       'numeric' = comma-separated integer (ex 1,200)
    #       'decimal' = comma-separated decimal to hundredths (ex 1,200.00)
    #       'dollar' = comma-separated whole number currency (USD) (ex $1,200)
    #       'dollar_cents' = comma-separated decimal currency (USD) to hundredths (ex $1,200.00)
    #       'percent' = integer percentage (ex 20%)
    #       'percent_1' = decimal percentage to tenths (ex 20.0%)
    #       'percent_2' = decimal percentage to hundredths (ex 20.00%)
        
    ## OPTIONAL:
    ### col_width is the width of the data columns. defaults to 14
    ### column_offset is the number of columns to shift to the right if you do not want your table to start on column A. defaults to 0

    # list of valid dtype args
    valid_dtypes = ['numeric','decimal_1','decimal_2','dollar','dollar_cents','percent','percent_1','percent_2','date','date_alt','datetime','datetime_alt']

    # this if statement sets the formatting based off the data_type argument
    ## it will raise an error to tell the user if they have entered an invalid data_type argument
    if data_type == 'numeric':
        data_format = wb.add_format({'num_format':'#,##0'})
    elif data_type == 'decimal_1':
        data_format = wb.add_format({'num_format':'#,##0.0'})
    elif data_type == 'decimal_2':
        data_format = wb.add_format({'num_format':'#,##0.00'})
    elif data_type == 'dollar':
        data_format = wb.add_format({'num_format':'$#,##0'})
    elif data_type == 'dollar_cents':
        data_format = wb.add_format({'num_format':'$#,##0.00'})
    elif data_type == 'percent':
        data_format = wb.add_format({'num_format':'0%'})
    elif data_type == 'percent_1':
        data_format = wb.add_format({'num_format':'0.0%'})
    elif data_type == 'percent_2':
        data_format = wb.add_format({'num_format':'0.00%'})
    elif data_type == 'date':
        data_format = wb.add_format({'num_format':'yyyy-mm-dd'})
    elif data_type == 'date_alt':
        data_format = wb.add_format({'num_format':'m/d/yyyy'})
    elif data_type == 'datetime':
        data_format = wb.add_format({'num_format':'yyyy-mm-dd h:mm'})
    elif data_type == 'datetime_alt':
        data_format = wb.add_format({'num_format':'m/d/yyyy h:mm AM/PM'})
    elif data_type == 'text':
        raise Exception('Data types are text by default! Function not needed.')
    else:
        raise ValueError(f"{data_type} is not a valid data_format option. Valid options are: {valid_dtypes}")

    # error if entered col_name not in dataframe

    # create list of all col_names
    col_name_list = [col_name for col_name in df.columns]

    if col_name not in col_name_list:
        raise ValueError(f"{col_name} not in dataframe. Columns in data are: {col_name_list}")
    else:
        pass

    
    # create list of all valid methods
    valid_methods = ['headers', 'data', 'all']

    # error if col_width_method not valid
    if col_width_method == None:
        pass
    elif col_width_method not in valid_methods:
        raise ValueError(f"{col_width_method} is not a valid method option, Valid methods are None or: {valid_methods}")
    else:
        pass

    # setting col_width

    # create an object holding the length of the name of the column
    ## + 1 for 'wiggle room'
    col_name_length = len(df.col_name.name) + 1

    # getting length of longest data point

    # list of all column values
    values = df[col_name].tolist()
    # create empty list to store the lengths
    value_lengths = []
    # iterating over the values list:
    for row_num, value in enumerate(values):
            # get the length in characters of each value
            length = len(str(value))
            # add it to the value_lengths list
            value_lengths.append(length)

    # getting row indices count of the data to use to set lower bound for formatting
    num_row_indices = len(df.index.names)
        
    # iterate through columns until we get to the selected column:
    for col_num, df_col_name in enumerate(df.columns):
        # if the specified column name matches 
        if df_col_name == col_name:
            sheet.set_column(col_num + num_row_indices + column_offset, col_num + num_row_indices + column_offset, col_width, data_format)
        else:
            pass
    

In [13]:
# create excel report

# nan_inf_to_errors will prevent nans from breaking report creation
## but any nans in the report are a bug that will need to be fixed

writer = pd.ExcelWriter('Reports/Example Clinical Report.xlsx', engine='xlsxwriter', options={'nan_inf_to_errors':True})

# exporting analyses to excel sheets
#twoD_df.to_excel(writer, sheet_name='Avg Svc Cost by Dept')

# creating workbook with sheets
ecr_wb = writer.book

twoD_sheet = ecr_wb.add_worksheet('Avg Svc Cost by Dept')

threeDrow_sheet1 = ecr_wb.add_worksheet('Clients by Dept & Gender')

threeDrow_sheet2 = ecr_wb.add_worksheet('Clients by Dept & Gender NEW')

test_dtype_sheet = ecr_wb.add_worksheet('Data Type Test')


# FORMATTING AVG SVC COST BY DEPT (2 dimensional data)

# header
last_col_highlight_header(twoD_df, ecr_wb, twoD_sheet, header_offset=2)

# index
format_index(twoD_df, ecr_wb, twoD_sheet, header_offset=2)

# data
insert_data(twoD_df, ecr_wb, twoD_sheet, header_offset=2, data_type='dollar_cents')

set_column_widths(twoD_df, ecr_wb, twoD_sheet)

# borders
table_bottom_border(twoD_df, ecr_wb, twoD_sheet, header_offset=2)

table_right_border(twoD_df, ecr_wb, twoD_sheet, header_offset=2)

# title
insert_title(twoD_df, ecr_wb, twoD_sheet, 'Avg Service Cost by Department')


# FORMATTING CLIENTS BY DEPT & GENDER (3 dimensional data with row mulitiindex)

# header
last_col_highlight_header(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

# index
merge_row_index_cells(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

format_row_multiindex(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

# data
insert_row_multiindex_data(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2, data_type='numeric')

set_column_widths(threeD_row_df_1, ecr_wb, threeDrow_sheet1)

# borders
table_bottom_border(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

table_right_border(threeD_row_df_1, ecr_wb, threeDrow_sheet1, header_offset=2)

# title
insert_title(threeD_row_df_1, ecr_wb, threeDrow_sheet1, 'Unique Clients by Department & Gender')


# FORMATTING CLIENTS BY DEPT & GENDER NEW (3 dimensional data with row mulitiindex)

# header
last_col_highlight_header(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

# index
merge_row_index_cells(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

format_row_multiindex(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

# data
insert_row_multiindex_data(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2, data_type='numeric')

set_column_widths(threeD_row_df_2, ecr_wb, threeDrow_sheet2)

# borders
table_bottom_border(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

table_right_border(threeD_row_df_2, ecr_wb, threeDrow_sheet2, header_offset=2)

# title
insert_title(threeD_row_df_2, ecr_wb, threeDrow_sheet2, 'Unique Clients by Department & Gender (New Gender Classification Method)')


# dtype test 
format_header(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
format_index(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
insert_data(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
set_column_widths(test_dtype_df, ecr_wb, test_dtype_sheet, method='all')

# drum rollllll
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'numeric', 'numeric')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'decimal_1', 'decimal_1')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'decimal_2', 'decimal_2')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'dollar', 'dollar')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'dollar_cents', 'dollar_cents')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'percent', 'percent')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'percent_1', 'percent_1')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'percent_2', 'percent_2')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'date', 'date')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'date_alt', 'date_alt')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'datetime', 'datetime')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'datetime_alt', 'datetime_alt')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'date_todt', 'date')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'date_alt_todt', 'date_alt')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'datetime_todt', 'datetime')
set_col_data_type(test_dtype_df, ecr_wb, test_dtype_sheet, 'datetime_alt_todt', 'datetime_alt')



table_bottom_border(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
table_right_border(test_dtype_df, ecr_wb, test_dtype_sheet, header_offset=2)
insert_title(test_dtype_df, ecr_wb, test_dtype_sheet, 'Data Type Test')

# save workbook
writer.save()
writer.close()

C:\Users\charl\AppData\Local\Temp\ipykernel_10608\1652452895.py:6: FutureWarning: Use of **kwargs is deprecated, use engine_kwargs instead.
  writer = pd.ExcelWriter('Reports/Example Clinical Report.xlsx', engine='xlsxwriter', options={'nan_inf_to_errors':True})
